In [1]:
#some basic imports and setups
import os
import numpy as np
from PIL import Image
import tensorflow as tf
import matplotlib.pyplot as plt


#mean of imagenet dataset in BGR
imagenet_mean = np.array([104., 117., 124.], dtype=np.float32)

current_dir = os.getcwd()
image_dir = os.path.join(current_dir, 'memes')
#image_dir = current_dir

%matplotlib inline

In [2]:
from alexnet import AlexNet

#placeholder for input and dropout rate
x = tf.placeholder(tf.float32, [1, 227, 227, 3])
keep_prob = tf.placeholder(tf.float32)

#create model with default config ( == no skip_layer and 1000 units in the last layer)
model = AlexNet(x, keep_prob, 1000,[],['fc7','fc8'],512,weights_path='/data/alpv95/MemeProject/im2txt/bvlc_alexnet.npy') #maybe need to put fc8 in skip_layers

#define activation of last layer as score
score = model.fc6

#create op to calculate softmax 
#softmax = tf.nn.softmax(score)

Converting captions and meme vector representations into single Tfrecord

Requires putting memes through alexnet to find their vector rep, shuffling the captions, changing captions into their word2idx, finally saving one caption together with one meme.

In [4]:
with open('/data/alpv95/MemeProject/im2txt/ordered_memes.txt','r') as f:
    img_files = f.readlines()
img_files = [os.path.join(image_dir, f) for f in img_files] # add path to each file
img_files = [img_file.replace('\n','') for img_file in img_files]
print(img_files[0],img_files[-1])
with open('/data/alpv95/MemeProject/im2txt/Captions.txt','r') as f:
    captions = f.readlines()
#captions = list(set(captions))
captions = [s.lower() for s in captions]
deleters = []
for i,capt in enumerate(captions):
    if ' - ' not in capt or ' - -' in capt:
        deleters.append(i)
for i,delete in enumerate(deleters):
    del captions[delete-i]
data_memes = []
data_captions = []
counter = 0

#Doing everything in one script: (the fc6 vectors are quite sparse)
with tf.Session() as sess:
    
    # Initialize all variables
    sess.run(tf.global_variables_initializer())
    
    # Load the pretrained weights into the model
    model.load_initial_weights(sess)
    
    for i,meme in enumerate(img_files):
        #meme_name = meme.replace('/Users/ALP/Desktop/Stanford/CS224n/MemeProject/memes/','')
        #meme_name = meme_name.replace('.jpg','').lower()
        #meme_name = meme_name.replace('-',' ')
        img = Image.open(meme)
        try:
            img.thumbnail((227, 227), Image.ANTIALIAS)
            #img = img.resize((227,227))
            #use img.thumbnail for square images, img.resize for non square
            assert np.shape(img) == (227, 227, 3)
        except AssertionError:
            img = img.resize((227,227))
            print('sizing error')
        
        # Subtract the ImageNet mean
        img = img - imagenet_mean #should probably change this
        
        # Reshape as needed to feed into model
        img = img.reshape((1,227,227,3))

        meme_vector = sess.run(score, feed_dict={x: img, keep_prob: 1}) #[1,4096]
        meme_vector = np.reshape(meme_vector,[4096])
        assert np.shape(meme_vector) == (4096,)
        #match = [s.split('-',1)[-1].lstrip() for s in captions if meme_name in s]
        
        match = []
        meme_name = captions[counter].split(' - ')[0]
        
        while meme_name in captions[counter]:
                if counter==len(captions)-1:
                    match.append(captions[counter].split(' - ')[-1])
                    break
                elif captions[counter] == captions[counter].split(' - ')[-1]:
                    counter += 1
                else:
                    match.append(captions[counter].split(' - ')[-1])
                    counter += 1
                
        
        #now save in tfrecords format, or prepare for that action
        meme_vectors = [meme_vector for cap in match]
        assert len(meme_vectors) == len(match)
        data_memes.extend(meme_vectors)
        data_captions.extend(match)

        if i % 100 == 0:
            print(i,len(data_memes),len(data_captions))
        
 

('/data/alpv95/MemeProject/im2txt/memes/y-u-no.jpg', '/data/alpv95/MemeProject/im2txt/memes/mendozameme.jpg')
(0, 194, 194)
(100, 16802, 16802)
sizing error
(200, 35976, 35976)
sizing error
sizing error
sizing error
(300, 55027, 55027)
sizing error
sizing error
(400, 74039, 74039)
sizing error
sizing error
(500, 92598, 92598)
sizing error
sizing error
(600, 111071, 111071)
sizing error
(700, 129455, 129455)
sizing error
(800, 147576, 147576)
sizing error
sizing error
sizing error
(900, 165608, 165608)
(1000, 183722, 183722)
sizing error
(1100, 201838, 201838)
(1200, 218644, 218644)
sizing error
(1300, 236360, 236360)
sizing error
sizing error
(1400, 252935, 252935)
sizing error
sizing error
sizing error
(1500, 269052, 269052)
(1600, 285491, 285491)
sizing error
sizing error
(1700, 301117, 301117)
sizing error
(1800, 315906, 315906)
sizing error
(1900, 330982, 330982)
(2000, 346091, 346091)
sizing error
sizing error
(2100, 361910, 361910)
(2200, 375182, 375182)
sizing error
sizing error

In [5]:
c = list(zip(data_memes, data_captions))
no_repeats = []
# order preserving
def idfun(x): return x

seen = {}
no_repeats = []
for item in c:
    marker = idfun(item[1])
    # in old Python versions:
    # if seen.has_key(marker)
    # but in new ones:
    if marker in seen: continue
    seen[marker] = 1
    no_repeats.append(item)
print(len(data_captions))
len(no_repeats)


414401


385708

In [6]:
from random import shuffle
shuffle(no_repeats)
memes_shuffled, captions_shuffled = zip(*no_repeats)

In [7]:
captions_shuffled[20]

'scored 4 times quick wasnt quick enough\n'

In [8]:
import re
word_captions = []
for capt in captions_shuffled:
    words = re.findall(r"[\w']+|[.,!?;'><(){}%$#£@-_+=|\/~`^&*]", capt)
    word_captions.append(words)
print(len(word_captions))
#word_captions = list(set(word_captions))
#print(len(word_captions))

385708


In [9]:
from collections import Counter
print("Creating vocabulary.")
counter = Counter()
for c in word_captions:
    counter.update(c)
print("Total words:", len(counter))

# Filter uncommon words and sort by descending count.
word_counts = [x for x in counter.items() if x[1] >= 3]
word_counts.sort(key=lambda x: x[1], reverse=True)
print("Words in vocabulary:", len(word_counts))

Creating vocabulary.
('Total words:', 150280)
('Words in vocabulary:', 41153)


In [10]:
# Create the vocabulary dictionary.
reverse_vocab = [x[0] for x in word_counts]
#unk_id = len(reverse_vocab)
vocab_dict = dict([(x, y) for (y, x) in enumerate(reverse_vocab)])

In [11]:
vocab_dict['hello']

365

In [14]:
EMBEDDING_DIMENSION=300 # Available dimensions for 6B data is 50, 100, 200, 300
data_directory = current_dir

PAD_TOKEN = 0

word2idx = { 'PAD': PAD_TOKEN } # dict so we can lookup indices for tokenising our text later from string to sequence of integers
weights = []
index_counter = 0

with open('glove.42B.300d.txt','r') as file:
    for index, line in enumerate(file):
        values = line.split() # Word and weights separated by space
        word = values[0] # Word is first symbol on each line
        if word in vocab_dict:
            index_counter += 1
            word_weights = np.asarray(values[1:], dtype=np.float32) # Remainder of line is weights for word
            word2idx[word] = index_counter # PAD is our zeroth index so shift by one
            weights.append(word_weights)
        if index % 100000 == 0:
            print(index)
        if index + 1 == 2000000:
            # Limit vocabulary to top 40k terms
            break

EMBEDDING_DIMENSION = len(weights[0])
# Insert the PAD weights at index 0 now we know the embedding dimension
weights.insert(0, np.random.randn(EMBEDDING_DIMENSION))

# Append unknown and pad to end of vocab and initialize as random #maybe include start and end token here
UNKNOWN_TOKEN=len(weights)
word2idx['UNK'] = UNKNOWN_TOKEN
word2idx['<S>'] = UNKNOWN_TOKEN + 1
word2idx['</S>'] = UNKNOWN_TOKEN + 2
weights.append(np.random.randn(EMBEDDING_DIMENSION)*0.5)
weights.append(np.random.randn(EMBEDDING_DIMENSION)*0.5)
weights.append(np.random.randn(EMBEDDING_DIMENSION)*0.5)

# Construct our final vocab
weights = np.asarray(weights, dtype=np.float32)

VOCAB_SIZE=weights.shape[0]
print(VOCAB_SIZE)

#Save Vocabulary
with tf.gfile.FastGFile('vocab3.txt', "w") as f:
    f.write("\n".join(["%s %d" % (w, c) for w, c in word2idx.iteritems()]))
print("Wrote vocabulary file:", 'vocab3.txt')

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
38520
('Wrote vocabulary file:', 'vocab3.txt')


In [15]:
weights[37984]

array([-0.44847  , -0.039212 ,  0.10393  , -0.18311  , -0.22794  ,
        0.071043 ,  0.68074  , -1.0702   ,  0.10879  ,  0.32581  ,
       -0.0075187, -0.7147   ,  0.38525  , -0.035042 ,  0.084247 ,
        0.21304  ,  0.025578 , -0.38933  , -0.43553  , -0.17755  ,
        0.4037   , -0.10913  , -0.056811 , -0.044236 , -0.12161  ,
       -0.37139  , -0.3668   , -0.78518  ,  0.1406   ,  0.41383  ,
       -0.06213  ,  0.64063  , -0.43111  , -0.16398  ,  0.15145  ,
        0.51573  ,  0.39917  , -0.4754   ,  0.24798  , -0.13286  ,
        0.30063  , -0.077097 , -0.064587 ,  0.30917  ,  0.65839  ,
        0.0059184,  0.10049  ,  0.34199  ,  0.16988  ,  0.15465  ,
        0.2108   , -0.16124  ,  0.16009  , -0.058738 ,  0.0602   ,
       -0.4676   , -0.41398  ,  0.13282  ,  0.34347  ,  0.31409  ,
       -0.2003   , -0.18261  ,  0.28005  ,  0.37784  , -0.085211 ,
       -0.82782  , -0.19752  , -0.16085  , -0.051853 , -0.046978 ,
       -0.090662 , -0.50244  ,  0.24386  ,  0.29688  , -0.2001

In [16]:
np.savetxt('embedding_matrix4',weights)

In [17]:
import re
token_captions = []
for capt in captions_shuffled:
    token_caption = []
    token_caption.append(word2idx['<S>'])
    words = re.findall(r"[\w']+|[.,!?;'><(){}%$#£@-_+=|\/~`^&*]", capt)
    for word in words:
        try:
            token = word2idx[word]
        except KeyError:
            token = word2idx['UNK']
        token_caption.append(token)
    token_caption.append(word2idx['</S>'])
    token_captions.append(token_caption)

In [18]:
print(token_captions[330000])
print(captions_shuffled[330000])
word2idx['UNK']

[38518, 1511, 318, 6315, 41, 113, 690, 4814, 37, 156, 30, 1043, 41, 113, 38519]
score against barcelona they said else terry will take your wife they said



38517

In [19]:
memes_shuffled = list(memes_shuffled)
captions_shuffled = list(captions_shuffled)

In [20]:
deleters = []
for i,ting in enumerate(token_captions):
    if len(ting) == 2:
        deleters.append(i)

In [21]:
for i,ting in enumerate(deleters):
    print(ting)
    del captions_shuffled[ting-i]
    del memes_shuffled[ting-i]
    del token_captions[ting-i]
deleters = []
for i,ting in enumerate(token_captions):
    if len(ting) == 2:
        deleters.append(i)
len(deleters)

76645
96449
123663
135465
185376
187451
233822
267491
327390
372006
374966
384502


0

In [22]:
many_unk = []
for i,capt in enumerate(token_captions):
    unk_counter = 0
    for token in capt:
        if token == word2idx['UNK']:
            unk_counter += 1
    if unk_counter >= 2:
        many_unk.append(i)
print(len(many_unk))

for i,ting in enumerate(many_unk):
    del captions_shuffled[ting-i]
    del memes_shuffled[ting-i]
    del token_captions[ting-i]

31996


In [23]:
len(memes_shuffled)

353700

In [24]:

def _int64_feature(value):
    """Wrapper for inserting an int64 Feature into a SequenceExample proto."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


def _bytes_feature(value):
    """Wrapper for inserting a bytes Feature into a SequenceExample proto."""
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


def _int64_feature_list(values):
    """Wrapper for inserting an int64 FeatureList into a SequenceExample proto."""
    return tf.train.FeatureList(feature=[_int64_feature(v) for v in values])


def _bytes_feature_list(values):
    """Wrapper for inserting a bytes FeatureList into a SequenceExample proto."""
    return tf.train.FeatureList(feature=[_bytes_feature(v) for v in values])

def _floats_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))

In [25]:
memes_shuffled_int = []
for i,meme in enumerate(memes_shuffled):
    memes_shuffled_int.append(np.int_(meme*1000000000000000))
print(memes_shuffled_int[20][:100])

[                0                 0                 0                 0
                 0                 0                 0                 0
                 0                 0  1886732697486877                 0
                 0                 0                 0                 0
                 0                 0                 0                 0
  5322175502777100                 0                 0                 0
                 0                 0  5142025470733643                 0
                 0 10490139007568360                 0                 0
  8445826530456543                 0   262790709733963                 0
                 0                 0                 0                 0
                 0                 0  5527552127838135                 0
                 0                 0                 0                 0
                 0                 0                 0                 0
                 0                 0               

In [26]:
(captions_shuffled[8])

'duct tape turning no into "hmmmnnmmmnn"\n'

In [27]:
import sys
train_filename = 'train.tfrecords6'  # address to save the TFRecords file
# open the TFRecords file
writer = tf.python_io.TFRecordWriter(train_filename)
for i in range(len(memes_shuffled_int)):
    if not i % 20000:
        print 'Train data: {}/{}'.format(i, len(memes_shuffled_int))
        sys.stdout.flush()
    context = tf.train.Features(feature={
          "train/meme": _bytes_feature(memes_shuffled_int[i].tostring()),  #this is the part that needs to be a float save
      })
    feature_lists = tf.train.FeatureLists(feature_list={
          "train/captions": _int64_feature_list(token_captions[i])
      })
    sequence_example = tf.train.SequenceExample(
          context=context, feature_lists=feature_lists)
    
    writer.write(sequence_example.SerializeToString())
    
writer.close()
sys.stdout.flush()

Train data: 0/353700
Train data: 20000/353700
Train data: 40000/353700
Train data: 60000/353700
Train data: 80000/353700
Train data: 100000/353700
Train data: 120000/353700
Train data: 140000/353700
Train data: 160000/353700
Train data: 180000/353700
Train data: 200000/353700
Train data: 220000/353700
Train data: 240000/353700
Train data: 260000/353700
Train data: 280000/353700
Train data: 300000/353700
Train data: 320000/353700
Train data: 340000/353700


In [126]:
num = 0
for capt in captions_shuffled:
    if 'what do we say to' in capt:
        #print(capt)
        num+=1
print(num)

489


In [152]:
x = list(enumerate(['a','b','c','d','e']))

In [153]:
x

[(0, 'a'), (1, 'b'), (2, 'c'), (3, 'd'), (4, 'e')]

In [154]:
for i,wp in enumerate(x):
    if wp[0] == 1:
        del x[i]
x = x[0:2]
print(x)

[(0, 'a'), (2, 'c')]


In [172]:
with tf.gfile.GFile('vocab2.txt', mode="r") as f:
      wordidx_pairs = list(f.readlines())
wordidx_pairs = [(line.split()[0],line.split()[1]) for line in wordidx_pairs]
vocab = dict([(x, int(y)) for (x, y) in wordidx_pairs]) #changed this to reflect vocab.txt format
x = sorted(vocab.iteritems(), key=lambda x: int(x[1]))
reverse_vocab = [y[0] for y in x]


In [177]:
type(vocab[','])

int